In [ ]:
#Run this cell to get SHAP characteristics for the dataset (OM0-OM8)
import shap
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import joblib
import os
import sys
import warnings


warnings.filterwarnings("ignore", category=UserWarning)

# ==================== CONFIGURATION ====================
DATA_DIR = r'File DIrectoty Address'----------> Put the directory address where all the necessary files are saved
MODEL_PATH = os.path.join(DATA_DIR, 'rf_model.pkl')
SCALER_PATH = os.path.join(DATA_DIR, 'rf_scaler.pkl')

def get_shap_features():
    # STRICT column ordering to ensure SHAP matches the model training
    return [
        'Data_1', 'Data_2', 
        'Data_3', 'Data_4', 'Data_5', 'Data_6', 
        'Data_7', 'Data_8', 'Data_9', 'Data10', 
        'Data_11', 'Data_12', 'Data_13', 
        'Data_14', 'Data_15', 'Data_16', 'Data_17'
    ]

# ==================== 1. LOAD RESOURCES ====================
print("[INFO] Loading Model, Scaler, and Data...")

if not os.path.exists(MODEL_PATH):
    print(f"[ERR] Model not found at {MODEL_PATH}")
    sys.exit()

model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# Load the data file with nominal sensor values (OM0) to serve as the background dataset
sample_file = os.path.join(DATA_DIR, 'OM0.xlsx')
if not os.path.exists(sample_file):
    print("[ERR] Could not find OM0.xlsx or Please ensure data exists.")
    sys.exit()

# Load and Preprocess
try:
    if sample_file.endswith('.xlsx'):
        df = pd.read_excel(sample_file)
    else:
        df = pd.read_csv(sample_file, sep=';', decimal=',')

    # Extract features in correct order
    feature_cols = get_shap_features()
    df_features = df[feature_cols].copy()

    # Scale data (Safe to ignore warning)
    X_sample = scaler.transform(df_features)

    # Create Display DataFrame (so plots show column names)
    X_display = pd.DataFrame(X_sample, columns=feature_cols)
    print(f"[INFO] Data loaded. Processing FULL dataset: {X_display.shape[0]} rows.")

except Exception as e:
    print(f"[ERR] Processing data failed: {e}")
    sys.exit()

# ==================== 2. CALCULATE SHAP (FULL DATA) ====================
print("[INFO] Calculating SHAP values for ALL rows (this may take time)...")

explainer = shap.TreeExplainer(model)

shap_values = explainer.shap_values(X_display)

is_list = isinstance(shap_values, list)
if not is_list:
    shap_array = np.array(shap_values)
    print(f"[DEBUG] SHAP output is an ARRAY of shape {shap_array.shape}")
else:
    print(f"[DEBUG] SHAP output is a LIST of length {len(shap_values)}")

# ==================== 3. GENERATE PLOTS (LOOP 0-8) ====================
print("[INFO] Generating plots for all 9 classes...")

for i in range(9):
    class_name = f"OM{i}"
    print(f"  > Processing {class_name}...")

    # Extract specific class data
    if is_list:
        if i >= len(shap_values): continue
        to_plot = shap_values[i]
    else:
        
        if len(shap_array.shape) == 3 and i < shap_array.shape[2]:
            to_plot = shap_array[:, :, i]
        else:
            to_plot = shap_array 

    # Plot
    plt.figure()
    plt.title(f"Feature Importance for {class_name} (Full Data)")
    
    shap.summary_plot(to_plot, X_display, show=False)
    
    # Save
    save_path = os.path.join(DATA_DIR, f'shap_summary_{class_name}.png')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

# ==================== 4. OVERALL IMPORTANCE BAR CHART ====================
print("  > Generating Overall Importance Bar Chart...")
plt.figure()

# Use Class 0 (Normal) data structure for the bar chart
baseline_data = shap_values[0] if is_list else (shap_array[:, :, 0] if len(shap_array.shape) == 3 else shap_array)

# Using X_display
shap.summary_plot(baseline_data, X_display, plot_type="bar", show=False)

plt.title("Overall Feature Importance Ranking (Full Data)")
plt.tight_layout()
plt.savefig(os.path.join(DATA_DIR, 'shap_importance_bar.png'), dpi=300)
plt.close()

print(f"\n[SUCCESS] All images saved to: {DATA_DIR}")

In [ ]:
# Run this cell just to train the model based on the sensor data files (OM files)
import os
import sys
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    classification_report
)
import warnings

warnings.filterwarnings('ignore')

# ==================== CONFIGURATION ====================
DATA_DIR = r'File DIrectoty Address'----------> Put the directory address where all the necessary files are saved
CLASS_NAMES = [f'OM{i}' for i in range(9)]
MAX_ROWS_PER_FILE = 1200

# ==================== 1. FEATURE SELECTION ====================
def get_shap_features():
    return [
        'Data_1', 'Data_2', 
        'Data_3', 'Data_4', 'Data_5', 'Data_6', 
        'Data_7', 'Data_8', 'Data_9', 'Data10', 
        'Data_11', 'Data_12', 'Data_13', 
        'Data_14', 'Data_15', 'Data_16', 'Data_17'
    ]

# ==================== 2. DATA LOADING ====================
def load_data():
    print(f"[DATA] Loading & Cleaning data...")
    X_list, y_list = [], []
    active_cols = get_shap_features()
    LAC_IDX, NIT_IDX = 0, 1

    for class_idx, class_name in enumerate(CLASS_NAMES):
        xlsx_path = os.path.join(DATA_DIR, f'{class_name}.xlsx')
        csv_path = os.path.join(DATA_DIR, f'{class_name}.csv')
        df = None
        try:
            if os.path.exists(csv_path):
                df = pd.read_csv(csv_path, sep=';', decimal=',')
            elif os.path.exists(xlsx_path):
                df = pd.read_excel(xlsx_path)
        except:
            pass
        if df is None:
            continue

        # Convert numeric-looking object columns
        for col in df.columns:
            if 'Time' not in col and df[col].dtype == 'object':
                try:
                    df[col] = (
                        df[col].astype(str)
                        .str.replace(',', '.')
                        .apply(pd.to_numeric, errors='coerce')
                    )
                except:
                    pass

        # Ensure all active columns exist
        for c in active_cols:
            if c not in df.columns:
                df[c] = 0.0

        df_filtered = df[active_cols].iloc[:MAX_ROWS_PER_FILE]
        X_data = df_filtered.values
        labels = np.full(len(df_filtered), class_idx)

        # CLEANING: If value is nominal, force label to 0
        if class_name in ['OM1', 'OM2', 'OM3', 'OM4']:
            vals = X_data[:, LAC_IDX]
            mask = (vals > 19.8) & (vals < 20.2)
            labels[mask] = 0
        elif class_name in ['OM5', 'OM6', 'OM7', 'OM8']:
            vals = X_data[:, NIT_IDX]
            mask = (vals > 3.9) & (vals < 4.1)
            labels[mask] = 0

        X_list.append(X_data)
        y_list.append(labels)
        print(f"  ✓ {class_name}: Loaded {len(labels)} rows")

    if not X_list:
        return None, None
    return np.vstack(X_list), np.hstack(y_list)

# ==================== 3. TRAINING & EVALUATION ====================
print("=" * 60 + "\n STARTING TRAINING \n" + "=" * 60)
X, y = load_data()

if X is not None:
    print("\n[SCALER] Fitting RobustScaler...")
    scaler = RobustScaler().fit(X)
    X_scaled = scaler.transform(X)
    joblib.dump(scaler, os.path.join(DATA_DIR, 'rf_scaler.pkl'))

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, shuffle=True
    )

    print("[TRAIN] Training Random Forest...")
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=15,
        min_samples_leaf=4,
        class_weight='balanced',
        random_state=42
    )
    rf.fit(X_train, y_train)

    joblib.dump(rf, os.path.join(DATA_DIR, 'rf_model.pkl'))

    # ===== METRICS =====
    preds = rf.predict(X_test)

    acc = accuracy_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)
    f1 = f1_score(y_test, preds, average='weighted')  

    print(f"\n>>> ACCURACY : {acc * 100:.2f}%")
    print(f">>> F1-score : {f1 * 100:.2f}%")
    print("\nConfusion matrix (raw values):\n", cm)
    print("\nClassification report:\n", classification_report(y_test, preds))

    # ===== PLOT & SAVE CONFUSION MATRIX =====
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=CLASS_NAMES,
        yticklabels=CLASS_NAMES
    )
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix (F1 = {f1*100:.2f}%)')
    plt.tight_layout()
    out_path = os.path.join(DATA_DIR, 'confusion_matrix_rf.jpg')
    plt.savefig(out_path, dpi=300)
    plt.close()
    print("Saved confusion matrix image to:", out_path)

    print("-" * 60)
else:
    print("[ERR] No data found.")


In [ ]:
#online fault detection mode
import os
import sys
import numpy as np
import pandas as pd
import json
import paho.mqtt.client as mqtt
from collections import deque, Counter
import joblib
import time
import warnings
import matplotlib.pyplot as plt  

warnings.filterwarnings('ignore')

# ==================== CONFIGURATION ====================
DATA_DIR = r'File DIrectoty Address'----------> Put the directory address where all the necessary files are saved
CLASS_NAMES = [f'OM{i}' for i in range(9)]
WARMUP_TIME = 10.0

# MQTT Settings
MQTT_BROKER = 'localhost'
MQTT_PORT = 1883
LIVE_FEATURES_TOPIC = 'fault_detection/live/features'
LIVE_RESULT_TOPIC = 'fault_detection/live/prediction'

# Topics required by your Simulink setup (Bridge)
TOPIC_CONTROL = 'fault_detection/live/control'
TOPIC_SOFT_SENSOR = 'fault_detection/live/soft_sensor'

# ==================== UTILS ====================
def get_shap_features():
    return [
        'Data_1', 'Data_2', 
        'Data_3', 'Data_4', 'Data_5', 'Data_6', 
        'Data_7', 'Data_8', 'Data_9', 'Data10', 
        'Data_11', 'Data_12', 'Data_13', 
        'Data_14', 'Data_15', 'Data_16', 'Data_17'
    ]

class SignalProcessor:
    """Smoothes input data for more stable detection."""
    def __init__(self, alpha=0.3):
        self.alpha = alpha
        self.smoothed_input = None

    def smooth_input(self, raw):
        raw_arr = np.array(raw)
        if self.smoothed_input is None: self.smoothed_input = raw_arr
        else: self.smoothed_input = (self.alpha * raw_arr) + ((1 - self.alpha) * self.smoothed_input)
        return self.smoothed_input.tolist()

# ==================== LIVE INTERFACE ====================
class LiveInterface:
    def __init__(self, model, scaler):
        self.model = model
        self.scaler = scaler
        self.client = mqtt.Client()
        self.filter = SignalProcessor(alpha=0.3)
        self.logs = []

    def on_connect(self, client, userdata, flags, rc):
        print("[MQTT] Connected to Live Interface. Monitoring...")
        client.subscribe(LIVE_FEATURES_TOPIC)

    # Plotting Function
    def plot_latency_histogram(self, df):
        if 'Latency_ms' not in df.columns: return

        latencies = df['Latency_ms'].dropna()
        if len(latencies) == 0: return

        mean_val = np.mean(latencies)
        median_val = np.median(latencies)
        p95_val = np.percentile(latencies, 95)
        
        plt.figure(figsize=(10, 6))
        
        # Standard bins 0-400ms 
        bins_list = np.arange(0, 401, 20) 
        plt.hist(latencies, bins=bins_list, color='#72a2c6', edgecolor='black', alpha=0.8)
        
        plt.axvline(mean_val, color='red', linestyle='dashed', linewidth=2, label=f'Mean = {mean_val:.2f} ms')
        plt.axvline(median_val, color='blue', linestyle='dashed', linewidth=2, label=f'Median = {median_val:.2f} ms')
        plt.axvline(p95_val, color='green', linestyle='dashed', linewidth=2, label=f'P95 = {p95_val:.2f} ms')
        
        plt.xlim(0, 400)
        
        plt.title(f'Detection Latency Distribution ({len(latencies):,} samples)', fontsize=14, fontweight='bold')
        plt.xlabel('Detection Latency (milliseconds)', fontsize=12)
        plt.ylabel('Frequency (count)', fontsize=12)
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.3)
        
        save_path = os.path.join(DATA_DIR, 'latency_histogram_detection.png')
        plt.savefig(save_path, dpi=300)
        print(f"[GRAPH] Latency histogram saved to: {save_path}")
        plt.close()

    def on_message(self, client, userdata, msg):
        try:
            # Start Timer
            t_start = time.time()
            
            payload = json.loads(msg.payload.decode())
            sim_time = float(payload.get("Time", 0))
            feat = payload.get("features", {})
            if not feat: return
            
            # Extract Raw Data for Soft Sensor display 
            raw_lac = float(feat.get('Data_1', 20.0))
            raw_ade = float(feat.get('Data_2', 4.0))

            if sim_time < WARMUP_TIME:
                print(f"T={sim_time:06.2f} | Status: WARMING UP...", end='\r')
                return

            # Prepare Features
            cols = get_shap_features()
            ai_input = [float(feat.get(c, 0)) for c in cols]
            
            # --- DETECTION LOGIC  ---
            smoothed = self.filter.smooth_input(ai_input)
            vec = self.scaler.transform(np.array(smoothed).reshape(1, -1))
            
            # Get Probabilities
            probs = self.model.predict_proba(vec)[0]
            pred = int(np.argmax(probs))
            vote_name = CLASS_NAMES[pred]
            
            # --- SIMULINK BRIDGE ---
            # 1. Publish Prediction
            self.client.publish(LIVE_RESULT_TOPIC, json.dumps({
                "fault_id": pred,
                "locked_id": 0 
            }))

            # 2. Publish Soft Sensors
            try:
                self.client.publish(TOPIC_SOFT_SENSOR, json.dumps({
                    "soft_lactose": round(smoothed[0], 4), 
                    "soft_adenine": round(smoothed[1], 4)
                }))
            except: pass

            # 3. Force Actuators to 0 
            self.send_ctrl(0.0, 0, "MONITORING", "AI_Lactose_Actuator")
            self.send_ctrl(0.0, 0, "MONITORING", "AI_Adenine_Actuator")
            
            # End Timer & Calculate Latency
            t_end = time.time()
            latency_ms = (t_end - t_start) * 1000

            # Display
            print(f"T={sim_time:06.2f} | Detected: {vote_name} (ID: {pred}) | Lat: {latency_ms:.2f}ms   ", end='\r')

            # --- LOGGING ---
            log_entry = {
                "Time": sim_time,
                "Final_Vote": pred,
                "Vote_Name": vote_name,
                "Latency_ms": latency_ms 
            }
            # Add separate columns for each confidence score
            for i, prob in enumerate(probs):
                log_entry[f"Conf_OM{i}"] = prob
                
            self.logs.append(log_entry)
            
            # --- SHUTDOWN & SAVE ---
            if sim_time >= 120.0:
                self.save_and_exit()

        except Exception as e: print(e)

    def send_ctrl(self, val, fid, reason, target):
        try:
            payload = {"action": "SET_PARAM", "target_block": target, "value": str(val), "fault_id": fid, "reason": reason}
            self.client.publish(TOPIC_CONTROL, json.dumps(payload))
        except: pass

    def save_and_exit(self):
        print("\n\n[INFO] Simulation finished. Saving Data & Graph...")
        
        # Convert list to DataFrame
        df = pd.DataFrame(self.logs)
        
        # Reorder columns: Time | Final_Vote | Vote_Name | Latency_ms | Conf_OM0...
        fixed_cols = ['Time', 'Final_Vote', 'Vote_Name', 'Latency_ms'] + [f'Conf_OM{i}' for i in range(9)]
        df = df[fixed_cols]
        
        # Save Excel
        save_path = os.path.join(DATA_DIR, 'detection_log.xlsx')
        df.to_excel(save_path, index=False)
        print(f"[SUCCESS] Excel saved to: {save_path}")
        
        # Save Graph
        self.plot_latency_histogram(df)
        
        self.client.disconnect()
        sys.exit(0)

    def start(self):
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.client.connect(MQTT_BROKER, MQTT_PORT, 60)
        print("="*50)
        print(" LIVE INTERFACE ACTIVE (Monitoring + Logging + Latency)")
        print("="*50)
        self.client.loop_forever()

if __name__ == "__main__":
    if os.path.exists(os.path.join(DATA_DIR, 'rf_model.pkl')):
        model = joblib.load(os.path.join(DATA_DIR, 'rf_model.pkl'))
        scaler = joblib.load(os.path.join(DATA_DIR, 'rf_scaler.pkl'))
        LiveInterface(model, scaler).start()
    else:
        print("[ERR] Model not found! Please run training first.")

In [ ]:
#Run this code to control the actuation based on the faults 
import os
import sys
import json
import numpy as np
import paho.mqtt.client as mqtt
import joblib
import pandas as pd
import psutil
import time
import warnings
import matplotlib.pyplot as plt
from collections import deque, Counter

warnings.filterwarnings('ignore')

# ==================== CONFIGURATION ====================
DATA_DIR = r'File DIrectoty Address'----------> Put the directory address where all the necessary files are saved
AGENTS_DIR = os.path.join(DATA_DIR, 'agents')
MQTT_BROKER = 'localhost'
MQTT_PORT = 1883

TOPIC_FEATURES = 'fault_detection/live/features'
TOPIC_PREDICTION = 'fault_detection/live/prediction'
TOPIC_CONTROL = 'fault_detection/live/control'
TOPIC_SOFT_SENSOR = 'fault_detection/live/soft_sensor' 

NOMINAL_DATA1 = 20.0 
NOMINAL_DATA2 = 4.0
FAULT_START_TIME = 10.0
DECISION_TIME = 30.0

# ==================== 1. AGENTS ====================
class FaultAgent:
    def __init__(self, agent_id, model_path):
        self.id = agent_id
        try: self.model = joblib.load(model_path)
        except: self.model = None

    def evaluate(self, features):
        if self.model is None: return {"agent_id": self.id, "is_match": False, "confidence": 0.0}
        probs = self.model.predict_proba(features)[0]
        return {"agent_id": self.id, "is_match": probs[1] > 0.5, "confidence": probs[1]}

class SystemCoordinator:
    def __init__(self, scaler):
        self.scaler = scaler
        self.agents = []
        self.load_agents()

    def load_agents(self):
        for i in range(9):
            path = os.path.join(AGENTS_DIR, f'agent_{i}.pkl')
            if os.path.exists(path): self.agents.append(FaultAgent(i, path))

    def consult_agents(self, raw_features):
        X = self.scaler.transform(np.array(raw_features).reshape(1, -1))
        reports = [agent.evaluate(X) for agent in self.agents]
        active = [r for r in reports if r['is_match']]
        if not active: return max(reports, key=lambda x: x['confidence'])['agent_id'], 0.0
        return max(active, key=lambda x: x['confidence'])['agent_id'], max(active, key=lambda x: x['confidence'])['confidence']

# ==================== 2. Exponentially Weighted Moving Average Filter ====================
class UltraHeavyEWMAFilter:
    def __init__(self, start_value=0.0):
        self.alpha = 0.001   
        self.x = start_value 

    def update(self, z):
        self.x = (self.alpha * z) + ((1 - self.alpha) * self.x)
        return self.x

# ==================== 3. CONTROL MANAGER ====================
class ControlManager:
    def __init__(self, mqtt_client):
        self.client = mqtt_client
        
        # Use List (Not Deque) to capture ALL votes from T10-T30
        self.vote_buffer = [] 
        
        self.locked_fault = None
        
        # Initialize Soft Sensors
        self.soft_lac = UltraHeavyEWMAFilter(start_value=NOMINAL_DATA1) 
        self.soft_ade = UltraHeavyEWMAFilter(start_value=NOMINAL_DATA2)
        
        # State Memory
        self.AI_Lactose_Actuator = 0.0
        self.AI_Adenine_Actuator = 0.0
        
        self.MAX_RATE_CHANGE = 0.2 

    def execute(self, sim_time, consensus_id, data1, data2):
        # 1. Update Filters
        pv_lac = self.soft_lac.update(data1)
        pv_ade = self.soft_ade.update(data2)

        # 2. Publish CLEAN Values
        try:
            self.client.publish(TOPIC_SOFT_SENSOR, json.dumps({
                "soft_lactose": round(pv_lac, 4),
                "soft_adenine": round(pv_ade, 4),
                "fault_id": int(consensus_id)
            }))
        except: pass

        # --- PHASE 1: OBSERVATION ---
        if sim_time < DECISION_TIME:
            # Only count votes if T >= 10.0 (Ignore start noise)
            if sim_time >= FAULT_START_TIME and consensus_id != 0: 
                self.vote_buffer.append(consensus_id)
            
            # Send Locked ID as 0 during observation
            self.send_pred(consensus_id, 0)
            self.send_ctrl(0.0, 0, "OBSERVING", "AI_Lactose_Actuator")
            self.send_ctrl(0.0, 0, "OBSERVING", "AI_Adenine_Actuator")
            return

        # --- PHASE 2: LOCKING ---
        if self.locked_fault is None:
            if self.vote_buffer:
                # Count all votes collected between T=10 and T=30
                self.locked_fault = Counter(self.vote_buffer).most_common(1)[0][0]
                print(f"\n[MANAGER] Locked on FAULT {self.locked_fault}\n")
            else:
                self.locked_fault = 0

        # Send the Locked ID 
        self.send_pred(consensus_id, self.locked_fault) 

        # --- PHASE 3: ACTUATION ---
        fid = self.locked_fault
        
        # --- BLOCK A: LACTOSE CONTROL (Faults 1, 2, 3, 4) ---
        if 1 <= fid <= 4:
            next_val = 0.0
            
            # === FAULT 4: NOISE ===
            if fid == 4:
                error = NOMINAL_DATA1 - pv_lac
                if abs(error) < 1.0:
                     next_val = 0.0 
                else:
                     Kp = 0.5 
                raw_act = error * Kp
                     delta = raw_act - self.AI_Lactose_Actuator
                     delta = np.clip(delta, -self.MAX_RATE_CHANGE, self.MAX_RATE_CHANGE)
                    next_val = self.AI_Lactose_Actuator + delta

            # === FAULT 3: PULSE ===
            elif fid == 3:
                
                if (30.0 <= sim_time < 60.0) or (80.0 <= sim_time < 120.0):
                    next_val = 20.0
                else:
                    next_val = 0.0

            # === FAULTS 1 & 2 ===
            else:
                dt = sim_time - FAULT_START_TIME
                if fid == 1: next_val = 4.0
                elif fid == 2: next_val = 0.2 * dt

            next_val = np.clip(next_val, -60, 60)
            self.AI_Lactose_Actuator = next_val
            self.AI_Adenine_Actuator = 0.0
            
            self.send_ctrl(self.AI_Lactose_Actuator, fid, "EXECUTING", "AI_Lactose_Actuator")
            self.send_ctrl(self.AI_Adenine_Actuator, fid, "LOCKED", "AI_Adenine_Actuator")


        # --- BLOCK B: ADENINE CONTROL (Faults 5, 6, 7, 8) ---
        elif 5 <= fid <= 8:
            next_val = 0.0
            
            if fid == 8:
                error = NOMINAL_DATA2 - pv_ade
                if abs(error) < 0.05:
                    next_val = 0.0
                else:
                    Kp = 1.0  
                    raw_act = error * Kp
                    delta = raw_act - self.AI_Adenine_Actuator
                    delta = np.clip(delta, -self.MAX_RATE_CHANGE, self.MAX_RATE_CHANGE)
                    next_val = self.AI_Adenine_Actuator + delta
            
            # === FAULT 7: TIME BASED ===
            elif fid == 7:
                if (30.0 <= sim_time < 60.0) or (80.0 <= sim_time < 120.0):
                    next_val = 4.0  
                else:
                    next_val = 0.0
            
            else:
                dt = sim_time - FAULT_START_TIME
                if fid == 5: next_val = -0.8
                elif fid == 6: next_val = -0.05 * dt

            next_val = np.clip(next_val, -60, 60)
            self.AI_Adenine_Actuator = next_val
            self.AI_Lactose_Actuator = 0.0
            
            self.send_ctrl(self.AI_Adenine_Actuator, fid, "EXECUTING", "AI_Adenine_Actuator")
            self.send_ctrl(self.AI_Lactose_Actuator, fid, "LOCKED", "AI_Lactose_Actuator")

        # --- BLOCK C: NO FAULT ---
        else:
            self.AI_Lactose_Actuator = 0.0
            self.AI_Adenine_Actuator = 0.0
            self.send_ctrl(0.0, 0, "NORMAL", "AI_Lactose_Actuator")
            self.send_ctrl(0.0, 0, "NORMAL", "AI_Adenine_Actuator")

    def send_pred(self, live_val, locked_val):
        msg = {
            "fault_id": int(live_val),
            "locked_id": int(locked_val)
        }
        self.client.publish(TOPIC_PREDICTION, json.dumps(msg))

    def send_ctrl(self, val, fid, reason, target):
        payload = {"action": "SET_PARAM", "target_block": target, "value": str(round(val, 4)), "fault_id": fid, "reason": reason}
        self.client.publish(TOPIC_CONTROL, json.dumps(payload))

# ==================== 4. MAIN BRIDGE ====================
class MAS_Bridge:
    def __init__(self):
        self.client = mqtt.Client()
        scaler_path = os.path.join(DATA_DIR, 'global_scaler.pkl')
        if not os.path.exists(scaler_path): raise Exception("Scaler missing.")
        self.scaler = joblib.load(scaler_path)
        
        self.coordinator = SystemCoordinator(self.scaler)
        self.manager = ControlManager(self.client)
        self.alpha = 0.3
        self.smooth_vec = None
        self.logs = []

    def on_connect(self, client, userdata, flags, rc):
        print(f"[MQTT] Connected. Agents listening...")
        client.subscribe(TOPIC_FEATURES)

    # [FIXED INDENTATION FOR PLOT FUNCTION]
    def plot_latency_histogram(self, df):
        if 'Latency_ms' not in df.columns: return

        latencies = df['Latency_ms'].dropna()
        if len(latencies) == 0: return

        mean_val = np.mean(latencies)
        median_val = np.median(latencies)
        p95_val = np.percentile(latencies, 95)
        
        plt.figure(figsize=(10, 6))
        plt.hist(latencies, bins=30, color='#72a2c6', edgecolor='black', alpha=0.8)
        
        plt.axvline(mean_val, color='red', linestyle='dashed', linewidth=2, label=f'Mean = {mean_val:.2f} ms')
        plt.axvline(median_val, color='blue', linestyle='dashed', linewidth=2, label=f'Median = {median_val:.2f} ms')
        plt.axvline(p95_val, color='green', linestyle='dashed', linewidth=2, label=f'P95 = {p95_val:.2f} ms')
        
        plt.title(f'Detection Latency Distribution ({len(latencies):,} samples)', fontsize=14, fontweight='bold')
        plt.xlabel('Detection Latency (milliseconds)', fontsize=12)
        plt.ylabel('Frequency (count)', fontsize=12)
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.3)
        
        save_path = os.path.join(DATA_DIR, 'latency_histogram.png')
        plt.savefig(save_path, dpi=300)
        print(f"[GRAPH] Latency histogram saved to: {save_path}")
        plt.close()

    
    def on_message(self, client, userdata, msg):
        try:
            t_start = time.time() # Start timer
            
            payload = json.loads(msg.payload.decode())
            sim_time = float(payload.get("Time", 0))
            feat = payload.get("features", {})
            if not feat: return

            cols = ['Data_1', 'Data_2', 'Data_3', 'Data_4', 'Data_5', 'Data_6', 'Data_7', 'Data_8', 'Data_9', 'Data_10', 'Data_11', 'Data_12', 'Data_13', 'Data_14', 'Data_15', 'Data_16', 'Data_17']
            raw = [float(feat.get(c, 0)) for c in cols]
            
            if self.smooth_vec is None: self.smooth_vec = np.array(raw)
            else: self.smooth_vec = (self.alpha * np.array(raw)) + ((1 - self.alpha) * self.smooth_vec)

            winner_id, confidence = self.coordinator.consult_agents(self.smooth_vec)
            self.manager.execute(sim_time, winner_id, raw[0], raw[1]) 
            
            # End Timer
            t_end = time.time()
            latency_ms = (t_end - t_start) * 1000

            # Debug
            locked_status = self.manager.locked_fault if self.manager.locked_fault is not None else 0
            print(f"[T={sim_time:06.1f}] Live:{winner_id} | Locked:{locked_status} | LacAct:{self.manager.AI_Lactose_Actuator:.2f} | Lat:{latency_ms:.2f}ms", end='\r')
            
            self.logs.append({
                "Time": sim_time, 
                "Fault": winner_id, 
                "AI_Lactose_Actuator": self.manager.AI_Lactose_Actuator, 
                "AI_Adenine_Actuator": self.manager.AI_Adenine_Actuator,
                "Latency_ms": latency_ms
            })
            
            if sim_time >= 120.0: self.shutdown()

        except Exception as e: print(f"\n[ERR] {e}")

    def shutdown(self):
        print("\n[MAS] Simulation finished. Saving data...")
        # Save Main Logs
        df_logs = pd.DataFrame(self.logs)
        df_logs.to_excel(os.path.join(DATA_DIR, 'mas_logs_final.xlsx'), index=False)
        
        # Use df_logs for plotting (MAS code uses self.logs, not self.control_log)
        self.plot_latency_histogram(df_logs)
        
        self.client.disconnect()
        sys.exit(0)

    def start(self):
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.client.connect(MQTT_BROKER, MQTT_PORT, 60)
        self.client.loop_forever()

if __name__ == "__main__":
    print("="*60)
    print(" CONTROL with FaultID based on T10 to T30 vote")
    print("="*60)
    bridge = MAS_Bridge()
    bridge.start()